In [1]:
import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Conv2DTranspose
import imageio
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import scipy.io as sio

Using TensorFlow backend.


In [2]:
# Turn inFile into a one hot label
def createOneHot(inFile,numberOfClasses):
    outFile = np.zeros((inFile.shape[0],inFile.shape[1], numberOfClasses));
    for i in range(0,inFile.shape[0]):
        for j in range(0,inFile.shape[1]):
            outFile[i,j,inFile[i,j]] = 1;
    return outFile;

In [15]:
# Turn 41 orthomosaics into one array. 
def returnOrthomosaicArray(imagePath1,imagePath2):
    temp= imageio.imread(imagePath1 + '1.png');
    inputData = np.zeros( (temp.shape[0],temp.shape[1],inputChannels) )
    for i in range(1,25):
        temp= imageio.imread(imagePath1 + str(i) + '.png');
        inputData[:,:,i] = temp[:,:,0];
    for i in range(1,16):
        temp= imageio.imread(imagePath2 + str(i) + '.png');
        inputData[:,:,i+25] = temp[:,:,0];
    return np.expand_dims(inputData,axis=0)

In [4]:
def makeDimensionsEven(inArray):
    if inArray.shape[0] % 2 != 0:
        middleArray = np.append(inArray, np.zeros((1,inArray.shape[1])), axis=0)
    else:
        middleArray = inArray
    if middleArray.shape[1] % 2 != 0:
        outArray = np.append(middleArray, np.zeros((middleArray.shape[0],1)), axis=1)
    else:
        outArray = middleArray;
    return outArray;

def loadData(pathArray):
    # each row of pathArray belongs to one dataset and has pathNIR, pathVIS, pathLabel
    #trainDataIn = np.load('trainingData.npy');
    outData = [];
    temp = returnOrthomosaicArray(pathArray[0,0],pathArray[0,1]);
    outData.append(temp) 

In [5]:
def loadLabel(path):
    temp = sio.loadmat(path);
    trainOneHotLabelsDim0 = np.expand_dims( createOneHot( temp['labeledPicture'], 9) ,axis=0);
    return trainOneHotLabelsDim0
    

model = Sequential([
    Conv2D(64, 5, input_shape=inputDataShape, activation='relu'),
    MaxPooling2D(),
    Conv2D(128, 5, activation='relu'),
    MaxPooling2D(),
    Conv2DTranspose(128, 6, strides=2, padding='valid', activation='relu'),
    Conv2DTranspose(64, 6, strides=2, activation='relu'),
    Conv2D(amountOfClasses, (1,1), activation='softmax')
])

In [6]:
imagePath1NIR = '/Volumes/mac_jannic_2017/thanujan/Datasets/Ximea_Tamron/20170622/Orthomosaics/Band';
imagePath1VIS = '/Volumes/mac_jannic_2017/thanujan/Datasets/Ximea_Tamron/20170622/VIS_Orthomosaics/Band';
labelpath1 = '/Volumes/mac_jannic_2017/thanujan/Datasets/labeledOrthomosaics/XimeaT20170622.mat';

imagePath2NIR = '/Volumes/mac_jannic_2017/thanujan/Datasets/FIP/20170622/testSet/Orthomosaics/Band';
imagePath2VIS = '/Volumes/mac_jannic_2017/thanujan/Datasets/FIP/20170622/testSet/VIS_Orthomosaics/Band';
labelpath2 = '/Volumes/mac_jannic_2017/thanujan/Datasets/labeledOrthomosaics/FIP20170622.mat';

amountOfClasses = 9;
inputChannels = 41;

model = Sequential([
    Conv2D(64, 5, input_shape=(None, None, inputChannels), activation='relu'),
    MaxPooling2D(),
    Conv2DTranspose(64, 6, strides=2, activation='relu'),
    Conv2D(amountOfClasses, (1,1), activation='softmax')
])

model.compile(optimizer='adagrad',
              loss='categorical_crossentropy',
              metrics=['accuracy']);

In [16]:
# Load the training data and labels

data1 = returnOrthomosaicArray(imagePath1NIR,imagePath1VIS)
#data1 = makeDimensionsEven(data1)
labels1 = loadLabel(labelpath1)
data2 = returnOrthomosaicArray(imagePath2NIR,imagePath2VIS)
labels2 = loadLabel(labelpath2)



In [17]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, None, None, 64)    65664     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, None, None, 64)    0         
_________________________________________________________________
conv2d_transpose_1 (Conv2DTr (None, None, None, 64)    147520    
_________________________________________________________________
conv2d_2 (Conv2D)            (None, None, None, 9)     585       
Total params: 213,769
Trainable params: 213,769
Non-trainable params: 0
_________________________________________________________________


In [20]:
from keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='acc', patience=2);

model.fit(data1, labels1, epochs=1,callbacks=[early_stopping]);
#model.fit(data2, labels2, epochs=1,callbacks=[early_stopping]);
model.fit(data1, labels1, epochs=1,callbacks=[early_stopping]);
#model.fit(data2, labels2, epochs=1,callbacks=[early_stopping]);


Epoch 1/1
1/1 [==============================] - 58s 58s/step - loss: 5.2892 - acc: 0.7056
Epoch 1/1
1/1 [==============================] - 65s 65s/step - loss: 5.1644 - acc: 0.7056


In [51]:
model.save('networkModels/FCN-None-None-conv5.h5')

In [139]:
a=np.ones((2,2))
b=np.zeros((3,3))
np.expand_dims(b,axis=2)
print(b.shape)
b= np.append(a,b,axis=2)
b

(3, 3)


AxisError: axis 2 is out of bounds for array of dimension 2

In [9]:
print(data1.shape)
print(data2.shape)
data2 = makeDimensionsEven(data2)
print(data2.shape)



(1082, 1406, 41)
(1085, 998, 41)


ValueError: all the input arrays must have same number of dimensions